In [1]:
import pymongo
import pandas as pd
import numpy as np

In [2]:
cliente=pymongo.MongoClient('mongodb://localhost:27017')  # cliente , la conexion por defecto
db=cliente.EmptiedSpain   # llamada a la base de datos


In [4]:
n_pueblo = 'ALBORNOS'
info_pueblo=db.emptiedSpain.find({'Pueblo_desde':n_pueblo})

In [5]:
df = pd.DataFrame(info_pueblo)

In [6]:
def cleanNans(dic):
    a = dic['coordinates']
    if np.nan in a: 
        dic['coordinates'] = [0,0]
    return dic

In [7]:
df.location = df.location.apply(cleanNans)

In [8]:
df.head()

,_id,nombre,location,distance,Entidad,Pueblo_desde
0,62286aba268013b076d10a1d,Carrefour Express,"{'type': 'Point', 'coordinates': [-3.6965673, ...",111.538560,supermercado,ALBORNOS
1,62286aba268013b076d10a1e,Carrefour Market,"{'type': 'Point', 'coordinates': [-3.6945615, ...",111.579454,supermercado,ALBORNOS
2,62286aba268013b076d10a1f,Supermercados La Plaza de Dia,"{'type': 'Point', 'coordinates': [-3.6931541, ...",111.745447,supermercado,ALBORNOS
3,62286aba268013b076d10a20,Condis,"{'type': 'Point', 'coordinates': [-3.6865764, ...",112.091530,supermercado,ALBORNOS
4,62286aba268013b076d10a21,Supermercados La Plaza de Dia,"{'type': 'Point', 'coordinates': [-3.690762, 4...",112.096348,supermercado,ALBORNOS


In [9]:
db.emptiedSpain.create_index([('location', '2dsphere')])


'location_2dsphere'

In [10]:
def find_near(geopoint, radio=1000):
    return db.emptiedSpain.find({'$and':[{'location':{
        '$near': {'$geometry': geopoint, '$maxDistance':radio}
    }},{'Entidad': 'Hospital'}]})

In [11]:
pueblos=db.emptiedSpain.find({'Entidad': 'Entidad de población'})

In [12]:
df_p = pd.DataFrame(pueblos)

In [13]:
i = 1050
a = df_p.location[i]
df_p.iloc[i]

_id                                                                       9249000100
Comunidad Autónoma                                                   Castilla y León
Provincia                                                                     Burgos
Municipio                                                         Palazuelos de Muñó
Entidad Singular de Población                                     PALAZUELOS DE MUÑÓ
Cobertura ≥ 30Mbps                                                              0.74
Cobertura redes fijas ≥ 100Mbps                                                  0.0
Cobertura 3G HSPA                                                                1.0
Cobertura 4G LTE                                                                 1.0
Weather_info                       En Palazuelos de Muñó, los veranos son cortos,...
pob_tot_muni                                                                   530.0
partido_gov_x                                                    

In [149]:
pd.DataFrame(find_near(a, radio=600000))

,_id,NOMBRE,DIRECCION,TELEFONO,CODMU,MUNICIPIOS,CODPROV,PROVINCIAS,COMUNIDADES,CODPOSTAL,NCAMAS,FINALIDAD_ASISITENCIAL,DEPENDENCIA_PATRIMONIAL,location,Entidad
0,420026.0,HOSPITAL VIRGEN DEL MIRÓN,"CTRA. DE LOGROÑO, 8",975220850.0,421736.0,Soria,42.0,SORIA,CASTILLA Y LEÓN,42005.0,0.0,GERIATRÍA Y/O LARGA ESTANCIA,COMUNIDAD AUTÓNOMA,"{'type': 'Point', 'coordinates': [-2.463960482...",Hospital
1,420011.0,HOSPITAL SANTA BÁRBARA,"Pº SANTA BÁRBARA, S/N",975234300.0,421736.0,Soria,42.0,SORIA,CASTILLA Y LEÓN,42005.0,0.0,GENERAL,SEGURIDAD SOCIAL,"{'type': 'Point', 'coordinates': [-2.471273765...",Hospital
2,420040.0,COMPLEJO ASISTENCIAL DE SORIA,"Pº. STA. BÁRBARA, S/N",975234300.0,421736.0,Soria,42.0,SORIA,CASTILLA Y LEÓN,42005.0,319.0,GENERAL,ENTIDADES PÚBLICAS,"{'type': 'Point', 'coordinates': [-2.472561225...",Hospital
3,90098.0,HOSPITAL SANTIAGO APÓSTOL,"CTRA. DE ORÓN, S/N",947349000.0,92194.0,Miranda de Ebro,9.0,BURGOS,CASTILLA Y LEÓN,9200.0,114.0,GENERAL,SEGURIDAD SOCIAL,"{'type': 'Point', 'coordinates': [-2.989475378...",Hospital
